In [ ]:
import pandas as pd
import json
import requests

In [ ]:

data = pd.read_csv("sales.csv")
data.columns = data.columns.str.strip()

In [ ]:
features = [
    'CustomerID',
    'NameStyle',
    'Title',
    'FirstName',
    'MiddleName',
    'LastName',
    'Suffix',
    'CompanyName',
    'EmailAddress',
    'Phone',
    'PasswordHash',
    'PasswordSalt',
    'rowguid',
    'ModifiedDate',
]

In [ ]:
import pandas as pd          # for DataFrame operations
import json                  # to serialize/deserialize JSON
import requests              # to make HTTP calls

# -------------------------------
# 1. Load your data
# -------------------------------
# If you’re pulling from SQL:
datos = pd.read_sql(query, conn)

# If instead you’ve saved to CSV:
# datos = pd.read_csv('sales.csv')

# -------------------------------
# 2. Define which raw columns to send
# -------------------------------
# These should match the columns your scoring script drops internally:
drop_cols = [
    "CustomerID", "rowguid", "ModifiedDate",
    "PasswordHash", "PasswordSalt"
]
# Build the list of features by excluding the dropped columns
features = [c for c in datos.columns if c not in drop_cols]

# -------------------------------
# 3. Validate that all needed columns exist
# -------------------------------
missing = [f for f in features if f not in datos.columns]
if missing:
    print("Las siguientes columnas faltan en tu DataFrame:")
    print(missing)
    raise SystemExit  # stop execution if any are missing

# -------------------------------
# 4. Filter DataFrame down to just those features
# -------------------------------
data_filtered = datos[features]

# -------------------------------
# 5. Convert to the JSON payload expected by the endpoint
# -------------------------------
#   - orient='records' gives a list of row-dicts, one per record
payload = {"data": data_filtered.to_dict(orient="records")}
data_json = json.dumps(payload)

# -------------------------------
# 6. Read the scoring URI from your uri.json file
# -------------------------------
with open("uri.json", "r", encoding="utf-8") as f:
    scoring_uri = json.load(f)["URI"][0]

# -------------------------------
# 7. Send the POST request
# -------------------------------
headers = {"Content-Type": "application/json"}
response = requests.post(scoring_uri, data=data_json, headers=headers)

# -------------------------------
# 8. Handle the response
# -------------------------------
if response.status_code == 200:
    # parse JSON; our scoring script returns {'predicted_dates': [...]}
    result = response.json().get("predicted_dates", [])
    
    # attach predictions back to the original DataFrame
    datos["predicted_date"] = pd.to_datetime(result).date
    
    # show the enriched DataFrame
    display(datos)
else:
    print(f"Error en la predicción [{response.status_code}]: {response.text}")
